In [2]:
import argparse
import os
import numpy as np
import math

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader, TensorDataset, Dataset
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

In [3]:
a= np.random.randint(90, 100, [5])
a.sort()
print(a)
a = np.eye(10)
print(a[0])

[90 92 98 98 99]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [9]:
x = np.load("data/xtrain32.npy")
y = np.load("data/ytrain.npy")
print(x[0])
print(x.shape)
print(y.shape)

FileNotFoundError: [Errno 2] No such file or directory: 'data/xtrain32.npy'

In [3]:
x = torch.Tensor(x)
y = torch.Tensor(y).to(torch.int8)

In [4]:

print(y[0])

tensor(5, dtype=torch.int8)


In [5]:
class CustomTensorDataset(Dataset):
    """TensorDataset with support of transforms.
    """
    def __init__(self, tensors, transform=None):
        assert all(tensors[0].size(0) == tensor.size(0) for tensor in tensors)
        self.tensors = tensors
        self.transform = transform

    def __getitem__(self, index):
        x = self.tensors[0][index]

        if self.transform:
            x = self.transform(x)

        y = self.tensors[1][index]

        return x, y

    def __len__(self):
        return self.tensors[0].size(0)

In [6]:
os.makedirs("images/cwganweb_disc5_rmsprop", exist_ok=True)


digit_embeddings = np.load("digit_embeddings.npy")

class Arguments():
  def __init__(self):
    self.n_epochs = 200
    self.batch_size = 64
    self.lr = 0.00005
    self.b1 = 0.5
    self.b2 = 0.999
    self.n_cpu = 8
    self.latent_dim = 100
    self.n_classes = 50
    self.img_size = 32
    self.channels = 1
    self.sample_interval = 1000
    self.embedding_size = 50
    self.n_generator = 5
    self.clip_value = 0.01

opt = Arguments()

img_shape = (opt.channels, opt.img_size, opt.img_size*2)

cuda = True if torch.cuda.is_available() else False


class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.label_emb = nn.Linear(opt.embedding_size, opt.embedding_size)

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(opt.latent_dim + opt.embedding_size, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, noise, labels):
        # Concatenate label embedding and image to produce input
        gen_input = torch.cat((self.label_emb(labels), noise), -1)
        img = self.model(gen_input)
        img = img.view(img.size(0), *img_shape)
        return img


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.label_embedding = nn.Linear(opt.embedding_size, opt.embedding_size)

        self.model = nn.Sequential(
            nn.Linear(opt.embedding_size + int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1),
        )

    def forward(self, img, labels):
        # Concatenate label embedding and image to produce input
        d_in = torch.cat((img.view(img.size(0), -1), self.label_embedding(labels)), -1)
        validity = self.model(d_in)
        return validity


# Loss functions
adversarial_loss = torch.nn.MSELoss()

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

# Configure data loader
dataloader = torch.utils.data.DataLoader(
    CustomTensorDataset(tensors = (x, y), transform = transforms.Compose(
            [transforms.ToPILImage(), transforms.Resize(opt.img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
       )),
    batch_size=opt.batch_size,
    shuffle=True,
)

#

# Optimizers
optimizer_G = torch.optim.RMSprop(generator.parameters(), lr=opt.lr)#, betas=(opt.b1, opt.b2))
optimizer_D = torch.optim.RMSprop(discriminator.parameters(), lr=opt.lr)#, betas=(opt.b1, opt.b2))

FloatTensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if cuda else torch.LongTensor

#numbers = np.random.randint(0, 49, [10])
#numbers = np.sort(numbers)
numbers = [1, 22, 23, 26, 29, 33, 35, 37, 42, 48]
print("Numbers", numbers)
def sample_image(n_row, batches_done):
    """Saves a grid of generated digits ranging from 0 to n_classes"""
    """does not use n_row"""
    # Sample noise
    z = Variable(FloatTensor(np.random.normal(0, 1, (10*10, opt.latent_dim))))
    # Get labels ranging from 0 to n_classes for n rows
    labels = np.array([num for _ in range(10) for num in numbers])
    gen_labels = Variable(FloatTensor(digit_embeddings[labels]))
    gen_imgs = generator(z, gen_labels)
    save_image(gen_imgs.data, "images/cwganweb_disc5_rmsprop/%d.png" % batches_done, nrow=10, normalize=True)


# ----------
#  Training
# ----------
total_d_loss = 0.0
total_g_loss = 0.0

for epoch in range(opt.n_epochs):
    for i, (imgs, labels) in enumerate(dataloader):

        batch_size = imgs.shape[0]


        # Configure input
        real_imgs = Variable(imgs.type(FloatTensor))
        labels = Variable(FloatTensor(digit_embeddings[labels]))

        # -----------------
        #  Train Generator
        # -----------------
        if i%opt.n_generator==0:
            optimizer_G.zero_grad()

            # Sample noise and labels as generator input
            z = Variable(FloatTensor(np.random.normal(0, 1, (batch_size, opt.latent_dim))))
            gen_labels = Variable(FloatTensor(digit_embeddings[np.random.randint(0, opt.n_classes, batch_size)]))
            # Generate a batch of images
            gen_imgs = generator(z, gen_labels)

            # Loss measures generator's ability to fool the discriminator
            validity = discriminator(gen_imgs, gen_labels)
            g_loss = -torch.mean(validity)
            total_g_loss += g_loss.item()

            g_loss.backward()
            optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------
        
        optimizer_D.zero_grad()

        # Loss for real images
        validity_real = discriminator(real_imgs, labels)

        # Loss for fake images
        validity_fake = discriminator(gen_imgs.detach(), gen_labels)

        # Total discriminator loss
        d_loss = -torch.mean(validity_real)+torch.mean(validity_fake) 

        total_d_loss += d_loss.item()

        d_loss.backward()
        optimizer_D.step()
        # Clip weights of discriminator
        for p in discriminator.parameters():
            p.data.clamp_(-opt.clip_value, opt.clip_value)
            
            
        if i%1000==0:
            print(
                "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
                % (epoch, opt.n_epochs, i, len(dataloader), total_d_loss/1000, total_g_loss/1000)
            )
            total_d_loss = 0.0
            total_g_loss = 0.0

        batches_done = epoch * len(dataloader) + i
        if batches_done % opt.sample_interval == 0:
            sample_image(n_row=20, batches_done=batches_done)

Numbers [1, 22, 23, 26, 29, 33, 35, 37, 42, 48]
[Epoch 0/200] [Batch 0/4688] [D loss: 0.000009] [G loss: -0.000044]
[Epoch 0/200] [Batch 1000/4688] [D loss: -58.517069] [G loss: -45.360132]
[Epoch 0/200] [Batch 2000/4688] [D loss: -2.980305] [G loss: -35.347118]
[Epoch 0/200] [Batch 3000/4688] [D loss: -0.780441] [G loss: -10.823278]
[Epoch 0/200] [Batch 4000/4688] [D loss: -0.362927] [G loss: -16.931922]
[Epoch 1/200] [Batch 0/4688] [D loss: -0.268649] [G loss: -11.936668]
[Epoch 1/200] [Batch 1000/4688] [D loss: -0.359299] [G loss: -17.154287]
[Epoch 1/200] [Batch 2000/4688] [D loss: -0.231548] [G loss: -14.114906]
[Epoch 1/200] [Batch 3000/4688] [D loss: -0.216784] [G loss: -13.302336]
[Epoch 1/200] [Batch 4000/4688] [D loss: -0.254355] [G loss: -13.053647]
[Epoch 2/200] [Batch 0/4688] [D loss: -0.227354] [G loss: -8.842430]
[Epoch 2/200] [Batch 1000/4688] [D loss: -0.320079] [G loss: -15.349561]
[Epoch 2/200] [Batch 2000/4688] [D loss: -0.281836] [G loss: -13.649983]
[Epoch 2/200] 

[Epoch 45/200] [Batch 2000/4688] [D loss: -0.656619] [G loss: 1.945401]
[Epoch 45/200] [Batch 3000/4688] [D loss: -0.929629] [G loss: 1.930957]
[Epoch 45/200] [Batch 4000/4688] [D loss: -1.096417] [G loss: 0.776754]
[Epoch 46/200] [Batch 0/4688] [D loss: -0.529594] [G loss: 2.170484]
[Epoch 46/200] [Batch 1000/4688] [D loss: -0.922741] [G loss: 1.746680]
[Epoch 46/200] [Batch 2000/4688] [D loss: -0.937242] [G loss: 1.620752]
[Epoch 46/200] [Batch 3000/4688] [D loss: -0.834447] [G loss: 1.460701]
[Epoch 46/200] [Batch 4000/4688] [D loss: -0.856994] [G loss: 2.247192]
[Epoch 47/200] [Batch 0/4688] [D loss: -0.569196] [G loss: 0.286404]
[Epoch 47/200] [Batch 1000/4688] [D loss: -0.708755] [G loss: -1.319800]
[Epoch 47/200] [Batch 2000/4688] [D loss: -0.609410] [G loss: -5.483584]
[Epoch 47/200] [Batch 3000/4688] [D loss: -0.456409] [G loss: -9.822556]
[Epoch 47/200] [Batch 4000/4688] [D loss: -0.533039] [G loss: -12.884999]
[Epoch 48/200] [Batch 0/4688] [D loss: -0.327561] [G loss: -10.50

[Epoch 90/200] [Batch 3000/4688] [D loss: -0.409737] [G loss: 2.462104]
[Epoch 90/200] [Batch 4000/4688] [D loss: -0.353880] [G loss: 2.435538]
[Epoch 91/200] [Batch 0/4688] [D loss: -0.364661] [G loss: 1.818955]
[Epoch 91/200] [Batch 1000/4688] [D loss: -0.384806] [G loss: 1.067616]
[Epoch 91/200] [Batch 2000/4688] [D loss: -0.424580] [G loss: 0.105927]
[Epoch 91/200] [Batch 3000/4688] [D loss: -0.453497] [G loss: 3.381473]
[Epoch 91/200] [Batch 4000/4688] [D loss: -0.433049] [G loss: 3.188691]
[Epoch 92/200] [Batch 0/4688] [D loss: -0.289220] [G loss: 2.200481]
[Epoch 92/200] [Batch 1000/4688] [D loss: -0.564208] [G loss: 2.777920]
[Epoch 92/200] [Batch 2000/4688] [D loss: -0.477192] [G loss: 2.207238]
[Epoch 92/200] [Batch 3000/4688] [D loss: -0.416857] [G loss: 2.831530]
[Epoch 92/200] [Batch 4000/4688] [D loss: -0.412143] [G loss: 1.043558]
[Epoch 93/200] [Batch 0/4688] [D loss: -0.205535] [G loss: 1.604242]
[Epoch 93/200] [Batch 1000/4688] [D loss: -0.382034] [G loss: 2.199224]
[

[Epoch 135/200] [Batch 2000/4688] [D loss: -0.314845] [G loss: 0.103336]
[Epoch 135/200] [Batch 3000/4688] [D loss: -0.516158] [G loss: -1.672189]
[Epoch 135/200] [Batch 4000/4688] [D loss: -0.423268] [G loss: -1.883338]
[Epoch 136/200] [Batch 0/4688] [D loss: -0.218471] [G loss: -0.414915]
[Epoch 136/200] [Batch 1000/4688] [D loss: -0.430220] [G loss: -0.358059]
[Epoch 136/200] [Batch 2000/4688] [D loss: -0.448017] [G loss: -2.645470]
[Epoch 136/200] [Batch 3000/4688] [D loss: -0.410012] [G loss: -1.637460]
[Epoch 136/200] [Batch 4000/4688] [D loss: -0.312750] [G loss: -1.378755]
[Epoch 137/200] [Batch 0/4688] [D loss: -0.286157] [G loss: -2.548769]
[Epoch 137/200] [Batch 1000/4688] [D loss: -0.505851] [G loss: -1.202388]
[Epoch 137/200] [Batch 2000/4688] [D loss: -0.361435] [G loss: -1.921941]
[Epoch 137/200] [Batch 3000/4688] [D loss: -0.391054] [G loss: -3.446549]
[Epoch 137/200] [Batch 4000/4688] [D loss: -0.363261] [G loss: -2.901456]
[Epoch 138/200] [Batch 0/4688] [D loss: -0.36

KeyboardInterrupt: 